In [335]:
import pandas as pd

In [336]:
dataframe = pd.read_csv('./stores_sales_forecasting.csv', encoding='ISO-8859-1')

In [337]:
colomns = dataframe.columns
print(colomns)

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')


J'ai remarqué que les données sont une jointure sur 3 tables. Donc j'ai séparé pour avoir une meilleur compréhension des données.
J'inclus les colonnes Sales, Quantity, Discount, Profit dans tout les dataframe pour les étudier sur ses variables.  

In [338]:
Order_dataframe = dataframe[['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Sales', 'Quantity', 'Discount', 'Profit']]
Customer_dataframe = dataframe[['Row ID','Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Sales', 'Quantity', 'Discount', 'Profit']]
Product_dataframe = dataframe[['Row ID','Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']]

# Préparation des données

## Product

In [339]:
distinct_categories = Product_dataframe['Category'].unique()
print(distinct_categories)

['Furniture']


In [340]:
Product_dataframe = Product_dataframe.drop(columns=['Category'])
dataframe = dataframe.drop(columns=['Category'])

In [341]:
unique_sub_categories = Product_dataframe['Sub-Category'].unique()
print(unique_sub_categories)

['Bookcases' 'Chairs' 'Tables' 'Furnishings']


### Étude des catégories

In [342]:
quantity_per_sub_category = Product_dataframe.groupby('Sub-Category')['Quantity'].sum()
print(quantity_per_sub_category)

Sub-Category
Bookcases       868
Chairs         2356
Furnishings    3563
Tables         1241
Name: Quantity, dtype: int64


In [343]:
profit_per_sub_category = Product_dataframe.groupby('Sub-Category')['Profit'].sum()
print(profit_per_sub_category)

Sub-Category
Bookcases      -3472.5560
Chairs         26590.1663
Furnishings    13059.1436
Tables        -17725.4811
Name: Profit, dtype: float64


In [344]:
highest_discount_sub_category = Product_dataframe.groupby('Sub-Category')['Discount'].mean()
print(highest_discount_sub_category)

Sub-Category
Bookcases      0.211140
Chairs         0.170178
Furnishings    0.138349
Tables         0.261285
Name: Discount, dtype: float64


Bookcases and Tables have the highest discount rates and thus, which could explain the negative profit. 

## Order

In [345]:
unique_ship_modes = Order_dataframe['Ship Mode'].unique()
print(unique_ship_modes)

['Second Class' 'Standard Class' 'First Class' 'Same Day']


In [346]:
Order_dataframe['Order Date'] = pd.to_datetime(Order_dataframe['Order Date'])
Order_dataframe['Ship Date'] = pd.to_datetime(Order_dataframe['Ship Date'])

dataframe['Order Date'] = pd.to_datetime(dataframe['Order Date'])
dataframe['Ship Date'] = pd.to_datetime(dataframe['Ship Date'])

In [347]:
quantity_per_ship_mode = Order_dataframe.groupby('Ship Mode')['Quantity'].sum()
print(quantity_per_ship_mode)

Ship Mode
First Class       1238
Same Day           453
Second Class      1569
Standard Class    4768
Name: Quantity, dtype: int64


In [348]:
profit_per_ship_mode = Order_dataframe.groupby('Ship Mode')['Profit'].sum()
most_profitable_ship_mode = profit_per_ship_mode
print(most_profitable_ship_mode)

Ship Mode
First Class        3066.9474
Same Day            797.3484
Second Class       4226.2614
Standard Class    10360.7156
Name: Profit, dtype: float64


In [349]:
# Extract month from Order Date
Order_dataframe['Order Month'] = Order_dataframe['Order Date'].dt.month

# Group by Order Month and count the number of orders
orders_per_month = Order_dataframe.groupby('Order Month')['Order ID'].count()

# Find the month with the highest number of orders
most_important_month = orders_per_month
print(f"The most important month for orders is: {most_important_month}")

The most important month for orders is: Order Month
1      81
2      63
3     140
4     138
5     150
6     145
7     164
8     124
9     282
10    187
11    319
12    328
Name: Order ID, dtype: int64


In [350]:
# Group by Order Month and calculate the mean discount
average_discount_per_month = Order_dataframe.groupby('Order Month')['Discount'].mean()

# Find the month with the highest average discount
most_discounted_month = average_discount_per_month
print(f"The month with the highest average discount is: {most_discounted_month}")

The month with the highest average discount is: Order Month
1     0.157037
2     0.178889
3     0.182143
4     0.163551
5     0.187267
6     0.191310
7     0.166463
8     0.183790
9     0.154468
10    0.190749
11    0.171379
12    0.173811
Name: Discount, dtype: float64


In [351]:
# Group by Order Month and sum the Profit
profit_per_month = Order_dataframe.groupby('Order Month')['Profit'].sum()

# Find the month with the highest profit
most_profitable_month = profit_per_month
print(f"The most profitable month is: {most_profitable_month}")

The most profitable month is: Order Month
1    -1944.2130
2      693.5796
3      771.9875
4     1460.3261
5     2302.2981
6      982.3847
7     1412.6846
8        4.0941
9     5460.0023
10   -3027.9321
11    3920.0007
12    6416.0602
Name: Profit, dtype: float64


## Customer

In [352]:
unique_segments = Customer_dataframe['Segment'].unique()
print(unique_segments)

['Consumer' 'Corporate' 'Home Office']


In [353]:
unique_countries = Customer_dataframe['Country'].unique()
print(unique_countries)

['United States']


In [354]:
Customer_dataframe = Customer_dataframe.drop(columns=['Country'])

In [355]:
profit_per_segment = Customer_dataframe.groupby('Segment')['Profit'].sum()
print(profit_per_segment)

Segment
Consumer       6991.0786
Corporate      7584.8158
Home Office    3875.3784
Name: Profit, dtype: float64


In [356]:
sales_per_state = Customer_dataframe.groupby('State')['Sales'].sum()
most_sales_state = sales_per_state.idxmax()
most_sales_value = sales_per_state.max()
print(f"The state with the most sales is: {most_sales_state} with sales of {most_sales_value}")

The state with the most sales is: California with sales of 156064.6015


## Dataframe

In [357]:
# Merge Product_dataframe and Order_dataframe on 'Row ID'
merged_dataframe = pd.merge(Product_dataframe, Order_dataframe, on='Row ID')

# Group by 'Sub-Category' and 'Ship Mode' and count the occurrences
ship_mode_per_category = merged_dataframe.groupby(['Sub-Category', 'Ship Mode']).size().unstack(fill_value=0)
print(ship_mode_per_category)

Ship Mode     First Class  Same Day  Second Class  Standard Class
Sub-Category                                                     
Bookcases              49         7            48             124
Chairs                 85        39           134             359
Furnishings           146        52           184             575
Tables                 47        21            61             190


In [358]:
categories_per_state = dataframe.groupby(['State', 'Sub-Category']).size().unstack(fill_value=0)
print(categories_per_state)

Sub-Category          Bookcases  Chairs  Furnishings  Tables
State                                                       
Alabama                       0       5            3       3
Arizona                       3      16           21       9
Arkansas                      1       2            5       1
California                   52     130          191      71
Colorado                      9      14           24       4
Connecticut                   2       3            6       2
Delaware                      4       2           10       2
District of Columbia          0       1            2       0
Florida                       8      19           47      11
Georgia                       5       6           19       2
Idaho                         0       1            3       2
Illinois                     10      38           57      18
Indiana                       0       7           13       3
Iowa                          0       1            2       1
Kansas                  

In [359]:
# Calculate the difference in days between Order Date and Ship Date
Order_dataframe['Shipping_Duration'] = (Order_dataframe['Ship Date'] - Order_dataframe['Order Date']).dt.days

# Extract the order type (CA or US) from the Order ID
Order_dataframe['Order_Type'] = Order_dataframe['Order ID'].str[:2]

# Group by Order Type and calculate the average shipping duration
average_shipping_duration = Order_dataframe.groupby('Order_Type')['Shipping_Duration'].mean()

print(average_shipping_duration)

Order_Type
CA    3.905421
US    3.968992
Name: Shipping_Duration, dtype: float64


### Shipping Duration Analysis

The average shipping duration for orders with the `Order_Type` "CA" is approximately 3.91 days, while for "US" it is approximately 3.97 days. This minimal difference in shipping times indicates that there is no significant variation between the two order types. Therefore, we cannot draw a definitive conclusion about the meaning of the first two letters in the `Order ID` based on shipping duration alone.


In [360]:
# Group by 'Segment' and 'Sub-Category' and sum the Sales
sales_per_segment_category = dataframe.groupby(['Segment', 'Sub-Category'])['Sales'].sum().unstack(fill_value=0)
print(sales_per_segment_category)

# Group by 'Segment' and 'Sub-Category' and sum the Profit
profit_per_segment_category = dataframe.groupby(['Segment', 'Sub-Category'])['Profit'].sum().unstack(fill_value=0)
print(profit_per_segment_category)

Sub-Category   Bookcases      Chairs  Furnishings      Tables
Segment                                                      
Consumer      68632.7290  172862.742    49620.046  99933.7950
Corporate     34005.9243   99140.878    25001.266  70871.7175
Home Office   12241.3430   56445.483    17083.852  36160.0195
Sub-Category  Bookcases      Chairs  Furnishings     Tables
Segment                                                    
Consumer     -4435.6382  13235.3319    7919.4227 -9728.0378
Corporate      638.4502   8344.6565    3508.2077 -4906.4986
Home Office    324.6320   5010.1779    1631.5132 -3090.9447


```markdown
# Key Takeaways from Data Exploration

1. **Customer Segments**:
    - The data includes three customer segments: Consumer, Corporate, and Home Office.
    - Profit distribution across segments shows that Corporate customers are the most profitable, followed by Consumer and Home Office.

2. **Product Categories**:
    - The dataset contains four main product sub-categories: Bookcases, Chairs, Tables, and Furnishings.
    - Chairs are the most profitable sub-category, while Tables have the highest negative profit, likely due to high discount rates.

3. **Sales and Profit by State**:
    - California is the state with the highest sales, amounting to approximately $156,064.60.
    - Sales and profit vary significantly across different states, with some states showing negative profits.

4. **Shipping Modes**:
    - There are four shipping modes: Second Class, Standard Class, First Class, and Same Day.
    - Standard Class is the most used shipping mode and also the most profitable.

5. **Monthly Trends**:
    - The month with the highest number of orders is December, followed by November.
    - The most profitable month is December, while January shows a significant negative profit.

6. **Discount Analysis**:
    - The average discount varies by month, with June having the highest average discount.
    - High discount rates on certain sub-categories, such as Tables and Bookcases, contribute to negative profits.

7. **Order and Shipping Analysis**:
    - The dataset includes detailed information on order dates, ship dates, and shipping modes.
    - Grouping by order month and shipping mode provides insights into sales and profit trends over time.

8. **Data Preparation**:
    - The data has been split into three main dataframes: Order_dataframe, Customer_dataframe, and Product_dataframe for better analysis.
    - Columns such as 'Category' have been dropped from the Product_dataframe to focus on sub-categories.
```

```markdown
Although I could include various visualizations to represent the data graphically, I chose not to focus on them because I personally prefer working with textual data. This approach allows me to concentrate on the detailed analysis and insights derived from the data without the distraction of visual elements.
```

In [361]:
columns_to_drop = ['Row ID', 'Order ID', 'Customer ID', 'Product ID', 'Customer Name', 'Product Name', 'Country', 'City', 'State', 'Postal Code']
dataframe = dataframe.drop(columns=columns_to_drop)
print(dataframe.head())

  Order Date  Ship Date       Ship Mode   Segment Region Sub-Category  \
0 2016-11-08 2016-11-11    Second Class  Consumer  South    Bookcases   
1 2016-11-08 2016-11-11    Second Class  Consumer  South       Chairs   
2 2015-10-11 2015-10-18  Standard Class  Consumer  South       Tables   
3 2014-06-09 2014-06-14  Standard Class  Consumer   West  Furnishings   
4 2014-06-09 2014-06-14  Standard Class  Consumer   West       Tables   

       Sales  Quantity  Discount    Profit  
0   261.9600         2      0.00   41.9136  
1   731.9400         3      0.00  219.5820  
2   957.5775         5      0.45 -383.0310  
3    48.8600         7      0.00   14.1694  
4  1706.1840         9      0.20   85.3092  


In [362]:
# Sort the dataframe by Order Date
dataframe = dataframe.sort_values(by='Order Date')
# Add Quarter column from the Order Date without the year
dataframe['Quarter'] = dataframe['Order Date'].dt.quarter

dataframe = dataframe.drop(columns=['Order Date', 'Ship Date'])

dataframe = dataframe.drop(columns=['Profit', 'Quantity'])

print(dataframe.tail(10))

# Define the split point (e.g., 80% of the data for training and 20% for testing)
split_point = int(len(dataframe) * 0.8)

# Split the data into training and testing sets
train_data = dataframe.iloc[:split_point]
test_data = dataframe.iloc[split_point:]

print(f"Training data: {train_data.shape}")
print(f"Testing data: {test_data.shape}")

# Save the training and testing data to CSV
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

           Ship Mode    Segment   Region Sub-Category      Sales  Discount  \
1288     First Class  Corporate     West       Chairs   340.7040      0.20   
197   Standard Class   Consumer  Central    Bookcases    78.8528      0.32   
504   Standard Class  Corporate     West  Furnishings     7.4000      0.00   
393   Standard Class   Consumer     West       Chairs   393.5680      0.20   
1128  Standard Class   Consumer     West  Furnishings    68.4600      0.00   
1619    Second Class   Consumer    South       Chairs   258.7500      0.00   
1617    Second Class   Consumer    South       Chairs  1207.8400      0.00   
1057  Standard Class  Corporate     West  Furnishings   101.1200      0.00   
1618    Second Class   Consumer    South       Chairs   300.9800      0.00   
186   Standard Class   Consumer     East    Bookcases   323.1360      0.20   

      Quarter  
1288        4  
197         4  
504         4  
393         4  
1128        4  
1619        4  
1617        4  
1057        4